In [1]:
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd
import math
import textstat
import numpy as np
import fnmatch

#nltk.install("all")

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

import nlp

from seg import Segmentation

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

# os.remove(path)
# if os.path.exists(path):
#    os.remove(path)

In [2]:
print('Esto se va a tardar...')
corpus= pd.read_csv('idata_with_metrics_oraciones.csv')
print('ya se cargo!')

Esto se va a tardar...
ya se cargo!


In [3]:
corpus.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6605046 entries, 0 to 6605045
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id_documento               int64  
 1   id_segmento                int64  
 2   text                       object 
 3   automa_readability_ind_or  float64
 4   dale_chall_readability_or  float64
 5   polysyllables_or           int64  
 6   count_monosyllables_or     int64  
 7   text_limpio                object 
 8   automa_readability_ind_cl  float64
 9   dale_chall_readability_cl  float64
 10  polysyllables_cl           int64  
 11  count_monosyllables_cl     int64  
 12  init_s                     int64  
 13  largo                      int64  
dtypes: float64(4), int64(8), object(2)
memory usage: 705.5+ MB


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 10 19:16:37 2020

@author: pablonicolasr
"""
#from dataset import generate
from cluster import cluster_ip #importa el metodo DBSCAN
from lxml import etree #guardar los nombres de los xml 


actual_directory = os.getcwd()  #te paras en el directorio actual

ruta = os.path.join(actual_directory, "mini_corpus") #declaras la ruta en donde esta el corpus
ruta_del_directorio = str(ruta) + "\\*.txt"  #pedis que solo vea los .txt del directorio
textos_originales = glob.glob(ruta_del_directorio) # Me produce una lista con los nombres de los archivos txt del corpus

#esto te arma la carpeta de los textos limpios
if not os.path.exists(str(actual_directory)+"\\cleaned_corpus"):
    os.mkdir(str(actual_directory)+'\\cleaned_corpus')
cleaned_corpus = str(actual_directory)+"\\cleaned_corpus"

# De esta manera le voy robo el nombre a los xml
path_xml = str(actual_directory) + "\\mini_corpus" + "\\*.xml"
xml_docs = glob.glob(path_xml)

#esto te arma la carpeta de los textos sospechosos

if not os.path.exists(str(actual_directory)+"\\sospechoso"):
    os.mkdir(str(actual_directory)+'\\sospechoso')
else:
    #print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(str(actual_directory)+"\\sospechoso")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory)+"\\sospechoso", archivo)
            os.remove(ruta_completa)
            #print(f"Eliminado: {ruta_completa}")


if not os.path.exists(str(actual_directory) + "\\xml\\output_xml"):
    os.mkdir(str(actual_directory) + "\\xml\\output_xml")
else:
    #print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(str(actual_directory) + "\\xml\\output_xml")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory) + "\\xml\\output_xml", archivo)
            os.remove(ruta_completa)
            #print(f"Eliminado: {ruta_completa}")


sospechosos_path = str(actual_directory) + "\\sospechoso\\"
output_xml = str(actual_directory) + "\\xml\\output_xml\\"

# Comienzo a leer el conjunto de textos
len(textos_originales)

min_samples=[4,10,100,1000]
eps=[0.1,0.01,0.001]
metodo=[] #hdbscan, dbscan
metric=[]
listas=[]
#para las metricas
import perfmeasures as p
plag_path, det_path = os.path.join(os.getcwd(), "xml", "xml_annotations"), os.path.join(os.getcwd(), "xml", "output_xml")



for ms in min_samples:
    print('Entro al min_samples', ms)
    for ep in eps:
        print('Entro al epsilon', ep)
        for i, archive in enumerate(textos_originales): 

            print("Numero de archivo: " + str(i+1))

            #f = open(archive, encoding="utf-8-sig")
            # Leo contenido del archivo
            #text = f.read()

            # Cierro el txt
            #f.close()

            # Nombre de Archivo
            filename = os.path.basename(archive)
            # Separo nombre de archivo y extension
            filename2, extension = os.path.splitext(filename)

            #Seg = Segmentation(text)
            # La variable data_text posee el texto completo, separado en párrafos
            #data_text = Seg.paraSegmentation()

            #data_list = []

            #lista_to_df = []

            #for j, segmento in enumerate(data_text):

             #   data_list.append([j, re.sub("\n", " ", segmento), len(re.sub("\n", " ", segmento))])

              #  lista_to_df.append(
               #     [j, filename, nlp.getnumOfPunctN(segmento), nlp.gettypeToken(segmento)]
                #)
            df=corpus[corpus['id_documento']==(i)].copy()
            data_list = df.iloc[:,[1,2,13]].copy()

            #df["getfleshReadingEase"] = df["getfleshReadingEase"] / df["getfleshReadingEase"].max()

            # Seleccionar los valores numéricos del df
            # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
            dataframe = df.iloc[:,[3,4,6,8,9,10,11]]
            # Aprendizaje No Supervisado con DBSCAN
            cluster = cluster_ip(dataframe,ms, "cosine",eps=ep)
            labels = cluster.labels_
            df["labels"] = labels


            realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
            clusterNum=len(set(labels))
            n_noise = list(labels).count(-1)

            #print("Estimated number of clusters: %d" % realClusterNum)
            #print("Estimated number of noise points: %d" % n_noise)
            counts = df["labels"].value_counts().to_dict()
            #print(counts)

            # Me sirve para guardar los casos de plagio y compararlos
            xml_name = os.path.basename(xml_docs[i])
            xml_name2, extension = os.path.splitext(xml_name)

            # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
            sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")

            # Los fragmentos que fueron clasificados como plagio
            # Se guardan en el archivo .txt
            # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
            # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
            for x in range(len(df["labels"])):
                l = df.iloc[x,14]
                if l == -1:
                   # print(data_list.iloc[x,1])
                   # print(x)
                    sherlock.write(data_list.iloc[x,1]+os.linesep)

            sherlock.close()

            document_el = etree.Element("document", reference = xml_name2 + ".txt")

            offset = []

            init = 0

            sum_length = []

            last = -1

            for y in range(len(df["labels"])):

                l = df.iloc[y,14]

                if l == -1:

                   # print(y)

                    if y == 0:

                        sum_length.append([init, data_list.iloc[y,2]])

                        #print(f"Case 0, y = 0")

                    elif y != 0 and (y == (last + 1) or len(sum_length) == 0):

                        sum_length.append([init, data_list.iloc[y,2]])

                        #print(f"Case 1, y == (last + 1)")

                    elif y != 0 and y != (last + 1):

                        this_length = sum([l[1] for l in sum_length])

                        feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))

                        sum_length.clear()

                        #print(sum_length)

                        sum_length.append([init, data_list.iloc[y,2]])

                       # print(f"Case 2, y != (last + 1)")

                    last = y

                if y == (len(df["labels"]) - 1) and len(sum_length) > 0:

                   # print("Entro2")

                    this_length = sum([l[1] for l in sum_length])

                    feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))


                init += data_list.iloc[y,2] + 1





                et = etree.ElementTree(document_el)
                with open(output_xml+xml_name, "wb") as fe:
                    et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)
        print('Se estan calculando las metricas')
        metricas=p.main(False, plag_path, "detected-plagiarism", det_path,"detected-plagiarism")
        lista=[metricas[0],metricas[1],metricas[2],metricas[3],ep,ms]
        listas.append(lista)
csv=pd.DataFrame(listas)


Entro al min_samples 4
Entro al epsilon 0.1
Numero de archivo: 1
Numero de archivo: 2
Numero de archivo: 3
Numero de archivo: 4
Numero de archivo: 5
Numero de archivo: 6
Numero de archivo: 7
Numero de archivo: 8
Numero de archivo: 9
Numero de archivo: 10
Numero de archivo: 11
Numero de archivo: 12
Numero de archivo: 13
Numero de archivo: 14
Numero de archivo: 15
Numero de archivo: 16
Numero de archivo: 17
Numero de archivo: 18
Numero de archivo: 19
Numero de archivo: 20
Numero de archivo: 21
Numero de archivo: 22
Numero de archivo: 23
Numero de archivo: 24
Numero de archivo: 25
Numero de archivo: 26
Numero de archivo: 27
Numero de archivo: 28
Numero de archivo: 29
Numero de archivo: 30
Numero de archivo: 31
Numero de archivo: 32
Numero de archivo: 33
Numero de archivo: 34
Numero de archivo: 35
Numero de archivo: 36
Numero de archivo: 37
Numero de archivo: 38
Numero de archivo: 39
Numero de archivo: 40
Numero de archivo: 41
Numero de archivo: 42
Numero de archivo: 43
Numero de archivo: 

 C:\Users\Usuario\Documents\Diplo\mento_plagio\Nueva\automaticPlagiarism_diplodatos-main\tp3_todo>python perfmeasures.py

In [5]:
new_names={0:'Plagdet Score',
           1:'Recall',
           2:'Precision',
           3:'Granularity',
           4:'epsilon',5
           :'min_samples'}
csv = csv.rename(columns=new_names)

In [6]:
csv.to_csv('metricas_de_cluster_oraciones.csv', index=False) #archivo 1

In [7]:
csv

,Plagdet Score,Recall,Precision,Granularity,epsilon,min_samples
0,0.002873,0.001914,0.141928,1.486799,0.100,4
1,0.028450,0.043314,0.080561,2.945584,0.010,4
2,0.033223,0.369231,0.057315,6.926576,0.001,4
3,0.004357,0.003390,0.134504,1.863302,0.100,10
4,0.033696,0.081684,0.069806,3.704739,0.010,10
5,0.033534,0.514187,0.056041,7.077215,0.001,10
6,0.046308,0.096571,0.085111,2.874113,0.100,100
7,0.035983,0.353200,0.057337,5.689116,0.010,100
8,0.047593,0.861610,0.046065,2.574039,0.001,100
9,0.051933,0.414529,0.061436,3.171479,0.100,1000
